In [1]:
import pandas as pd
import numpy as np
from datetime import date

import pandas_datareader.data as web
import pandas_datareader as pdr

/Users/markhendricks/Projects/drw-markets-2023/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
START_DATE = '1999-01-01'
END_DATE = '2023-07-15'
TICKS = ['DGS5','DFII5']
LABS = ['Note (5yr)','TIPS (5yr)']

map_labels = {TICKS[i]:LABS[i] for i in range(len(TICKS))}
yields = pdr.DataReader(TICKS,data_source='fred', start=START_DATE,end=END_DATE).rename(columns=map_labels).dropna()
yields.index.name = 'date'
yields /= 100

In [3]:
TICK_P = 'CPIAUCSL'
LAB_P = 'CPI'

cpi = pdr.DataReader(TICK_P,data_source='fred', start=START_DATE,end=END_DATE).rename(columns={TICK_P:LAB_P}).dropna()
# FRED reports monthly as first of month instead of last of month
cpi = cpi.resample('M').last()
cpi.index.name = 'date'

In [4]:
info = pd.DataFrame(columns=['Description'],index=yields.columns)

info.iloc[0] = 'Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity, Quoted on an Investment Basis (DGS5)'
info.iloc[1] = 'Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity, Quoted on an Investment Basis, Inflation-Indexed (DFII5)'
info.loc[LAB_P] = 'Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL)'
info.index.name = 'Label'

# Save to Excel

In [5]:
with pd.ExcelWriter('../data/inflation_expectation_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name = 'descriptions')
    yields.to_excel(writer, sheet_name='yields')
    cpi.to_excel(writer, sheet_name='consumer prices')